In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Badwater 267 VR 2025/gpx.csv', low_memory=False)
df.head(1)

In [ ]:
meta_df = pd.read_csv('/content/drive/MyDrive/Badwater 267 VR 2025/runsignup.csv', low_memory=False)
meta_df.head(1)

In [ ]:
merged_df = df.merge(meta_df[['name', 'strava_id']], on='strava_id', how='inner')
merged_df.head(1)

In [ ]:
counts = merged_df.groupby(['strava_id', 'name']).size().reset_index(name='point_count')

Q1 = counts['point_count'].quantile(0.25)
Q3 = counts['point_count'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

counts['is_outlier'] = (counts['point_count'] < lower_bound) | (counts['point_count'] > upper_bound)
counts_sorted = counts.sort_values(by='point_count', ascending=False)

counts_sorted[counts_sorted['is_outlier'] == True].head(5)